# Start: Analysiere Daten

In [2]:
# import initial_data.csv
import pandas as pd
import numpy as np

from feasibility import is_feasible

# import data from csv file
def import_data():
    data = pd.read_csv('initial_data.csv')
    return data

data = import_data()

In [3]:
# put column names into a list
column_names = data.columns.values
print(column_names)
inputs = column_names[0:8]
print(inputs)
outputs = column_names[8:13]
print(outputs)

['Engine speed' 'Engine load' 'Railpressure' 'Air supply' 'Crank angle'
 'Intake pressure' 'Back pressure' 'Intake temperature' 'NOx' 'PM 1' 'CO2'
 'PM 2' 'Pressure cylinder']
['Engine speed' 'Engine load' 'Railpressure' 'Air supply' 'Crank angle'
 'Intake pressure' 'Back pressure' 'Intake temperature']
['NOx' 'PM 1' 'CO2' 'PM 2' 'Pressure cylinder']


In [4]:
def get_input_data(row):
    x1, x2, x3, x4, x5, x6, x7, x8 = data.iloc[row, 0], data.iloc[row, 1], data.iloc[row, 2], data.iloc[row, 3], data.iloc[row, 4], data.iloc[row, 5], data.iloc[row, 6], data.iloc[row, 7]
    return x1, x2, x3, x4, x5, x6, x7, x8

def get_output_data(row):
    x1, x2, x3, x4, x5 = data.iloc[row, 8], data.iloc[row, 9], data.iloc[row, 10], data.iloc[row, 11], data.iloc[row, 12]
    return x1, x2, x3, x4, x5

def get_critical_output_data(i):
    x1, x2, x3 = data.iloc[i, 9], data.iloc[i, 11], data.iloc[i, 12]
    return x1, x2, x3

In [5]:
# show if data row is feasible via the first 8 columns and the function is_feasible
feasible = []
for i in range(len(data)):
    x1, x2, x3, x4, x5, x6, x7, x8 = get_input_data(i)
    # print(x1, x2, x3, x4, x5, x6, x7, x8)
    feasible.append(is_feasible(x1, x2, x3, x4, x5, x6, x7, x8))
data['feasible'] = feasible

In [6]:
# Check if outputs are in safe range
# PM 1 < 6, PM 2 < 16, Pressure cylinder < 160
safe = []
for i in range(len(data)):
    x9, x10, x11 = get_critical_output_data(i)
    if x9 < 6 and x10 < 16 and x11 < 160:
        safe.append(True)
    else:
        safe.append(False)
data['safe'] = safe

In [7]:
# show unsafe data (safe = False)
unsafe_data = data[data['safe'] == False]
print(unsafe_data)

    Engine speed  Engine load  Railpressure  Air supply  Crank angle  \
70        1800.0        35.66        1521.7    310.5600         5.69   
89        2000.0       139.54        2500.3   1008.3008         3.66   

    Intake pressure  Back pressure  Intake temperature         NOx      PM 1  \
70           1123.5         1236.1                60.9   66.869420  4.095885   
89           3129.9         3329.4                79.7  328.147428  4.017909   

           CO2       PM 2  Pressure cylinder  feasible   safe  
70   36.291467  16.283255          60.497930      True  False  
89  163.080520   4.692198         160.047375      True  False  


In [8]:
# predict when to get safe data and when to get unsafe data
# PM 1 < 6, PM 2 < 16, Pressure cylinder < 160

In [5]:
import csv
import re

def read_csv_file(file_path):
    # Initialisiere eine leere Liste, um die bereinigten Daten zu speichern
    cleaned_data = []

    # Lese die CSV-Datei
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')  # Annahme: Semikolon als Trennzeichen
        for row in reader:
            # Verbinde die Zeilenelemente mit einem Komma, um das Trennzeichen zu vereinheitlichen
            unified_row = ','.join(row)
            # Ersetze mehrere aufeinander folgende Kommas durch ein einzelnes Komma
            unified_row = re.sub(r',+', ',', unified_row)
            # Teile die vereinheitlichte Zeile nach dem Komma auf
            split_row = unified_row.split(',')
            # Entferne die ersten zwei Parameter
            cleaned_row = split_row[1:]
            # Füge die bereinigte Zeile der Liste hinzu
            cleaned_data.append(cleaned_row)

    return cleaned_data

# Beispiel: Daten aus "input.csv" einlesen
input_file_path = "C:/Users/CM3EXN/dev/force-push/querys_ForcePush (1).csv"
cleaned_data = read_csv_file(input_file_path)

# Bereinigte Daten ausgeben
for row in cleaned_data:
    print(row)


['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature', 'NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder', 'costs']
['700', '7.33', '500.9', '78.3753', '1.91', '967.5', '1027.2', '65.5', '15.851380240572826', '0.13141209808324952', '2.9288776008636166', '2.2285245004540064', '43.95540914698391', '1']
['1868.56556485', '70.22861855', '2006.71078088', '713.87079231', '0.686327', '2155.33490657', '1955.29547907', '58.08805416', '299.3046013141297', '2.0448786460859996', '71.06901928396269', '0.04474019224655955', '110.99892178150688', '1']
['1997.8121513224614', '112.91547887925219', '1645.603139454999', '882.6126624300058', '5.033640047671298', '3007.3658892579892', '2534.258979984933', '72.51886978878605', '329.17019939060305', '4.9436483453137035', '139.07088250811285', '5505.047499557287', '147.62142320789607', '2']
['816.4712565906533', '37.609734587256995', '900.1427871055062', '228.42045796100746', '-8.5